In [1]:
!python3 -m ipykernel install --user --name=circompairing

Installed kernelspec circompairing in /Users/nibnalin/Library/Jupyter/kernels/circompairing


In [2]:
%pip install py_ecc

You should consider upgrading via the '/Users/nibnalin/.pyenv/versions/3.9.0/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from py_ecc.fields import (
    bn128_FQ as FQ,
    bn128_FQ2 as FQ2,
    bn128_FQ12 as FQ12,
)
from py_ecc.bn128 import (
    bn128_curve as curve
)
from field_helper import (
    numberToArray,
    numberToBase,
    hamming_weight,
    printEllipticPoint,
    printFQ,
    printFQ2,
    Fp12convert,
    printFQ12,
    print_fq12_frobenius_coeff
)
import math

In [27]:
from py_ecc.fields import (
    bn128_FQ as FQ,
    bn128_FQ2 as FQ2,
    bn128_FQ12 as FQ12,
)
from py_ecc.bn128 import (
    bn128_curve as curve,
    bn128_pairing as pairing
)
import json
with open('fixtures/vkey-0.json', 'r') as vkey_file:
    vkey_data = vkey_file.read()
vkey = json.loads(vkey_data)

x, y, z = tuple([FQ((int(x))) for x in vkey["vk_alpha_1"]]) 
negalpha = ( x / z, -(y / z) )
# print("negalpha", negalpha)

x, y, z = tuple([ FQ2([int(x[0]), int(x[1])]) for x in vkey["vk_beta_2"]])
beta = ( x / z, y / z )

# print("beta", beta)


x, y, z = tuple([ FQ2([int(x[0]), int(x[1])]) for x in vkey["vk_gamma_2"]])
gamma = ( x / z, y / z )

# print("gamma", gamma)


x, y, z = tuple([ FQ2([int(x[0]), int(x[1])]) for x in vkey["vk_delta_2"]])
delta = ( x / z, y / z )

# print("delta", delta)

public_input_count = vkey["nPublic"]

ICs = []
for i in range(public_input_count + 1):
    x, y, z = tuple([ FQ(int(x)) for x in vkey["IC"][i]])
    ICs.append( ( x / z, y / z ) )

negalphabeta = pairing.pairing( beta, negalpha )
#print("negalphabeta", negalphabeta)

def Fpconvert(X, n, k):
    return numberToArray(X.n, n, k)

def Fp2convert(X, n, k):
    return [ numberToArray(X.coeffs[0].n, n, k) , numberToArray(X.coeffs[1].n, n, k) ]

def Fp12convert(X, n, k):
    basis1 = X.coeffs
    ret = []
    for i in range(6):
        fq2elt = FQ2([basis1[i].n, 0]) + FQ2([basis1[i+6].n, 0]) * FQ2([9,1])
        ret.append(Fp2convert(fq2elt, n, k))
    return ret

n = 43
k = 6

inputParameters = {
    "gamma2": [ Fp2convert(gamma[0], n, k), Fp2convert(gamma[1], n, k)],
    "delta2": [ Fp2convert(delta[0], n, k), Fp2convert(delta[1], n, k)],
    "negalfa1xbeta2": Fp12convert(negalphabeta, n, k),
    "IC": [[Fpconvert(IC[0], n, k), Fpconvert(IC[1], n, k)] for IC in ICs],
}

print("inputParameters", inputParameters)

inputParameters {'gamma2': [[[5896345417453, 4240670514135, 6172078461917, 219834884668, 2138480846496, 206187650596], [6286472319682, 5759053266064, 8549822680278, 8639745994386, 912741836299, 219532437284]], [[4404069170602, 525855202521, 8311963231281, 825823174727, 854139906743, 161342114743], [3147424765787, 7086132606363, 7632907980226, 5320198199754, 6592898451945, 77528801456]]], 'delta2': [[[7564227285875, 7283380019653, 3682266449365, 4984321143902, 6583509475591, 327853737337], [3320972149216, 3443901704995, 4744965966124, 7685675311813, 5908322089317, 25577108242]], [[6093773101658, 2647216954316, 2960160419428, 6138008725045, 4717326821320, 2354926030], [7363987589847, 3368775245686, 5174436894138, 1606318281463, 5768751342308, 285222487834]]], 'negalfa1xbeta2': [[[4063420080633, 6555003798509, 3528875089017, 5800537096256, 8041381108016, 203518374640], [7676269984398, 1145806392863, 6738515895690, 5144301275423, 8547057760405, 353834589854]], [[5712635615088, 876347569869

In [28]:
with open('fixtures/proof-0.json', 'r') as proof_file:
    proof_data = proof_file.read()
proof = json.loads(proof_data)

x, y, z = tuple([FQ((int(x))) for x in proof["pi_a"]]) 
negpi_a = (x / z, - (y / z))

x, y, z = tuple([ FQ2([int(x[0]), int(x[1])]) for x in proof["pi_b"]])
pi_b = (x / z, y / z)

x, y, z = tuple([FQ((int(x))) for x in proof["pi_c"]]) 
pi_c = (x / z, y / z)

proofParameters = {
    "negpa": [Fpconvert(negpi_a[0], n, k), Fpconvert(negpi_a[1], n, k)],
    "pb": [ Fp2convert(pi_b[0], n, k), Fp2convert(pi_b[1], n, k)],
    "pc": [Fpconvert(pi_c[0], n, k), Fpconvert(pi_c[1], n, k)],
}

print("proofParameters", proofParameters)

proofParameters {'negpa': [[715750443755, 6436079997396, 3617168051710, 3426259975516, 5155153559542, 330687916410], [2201844111436, 6118550145056, 8204097730434, 1698101694170, 2980896029588, 301789638178]], 'pb': [[[852168095613, 970376859129, 8276645667705, 1820463407287, 2702364426581, 365678717349], [6787903672652, 27970718557, 736841611078, 5634029812080, 3671063362484, 254087494396]], [[6666704541219, 5831373363858, 2612370467409, 352875408002, 786375640946, 4114738810], [2077678317377, 3857154881586, 2480488762052, 7726672979522, 814426780610, 273260664015]]], 'pc': [[1217057748590, 4468333141377, 3428266047894, 7836784644393, 8023131878955, 234519486450], [6501109523029, 1795134812132, 2762455515554, 3291517429621, 908503686313, 32977388466]]}


In [29]:
with open('fixtures/public-0.json', 'r') as public_file:
    public_data = public_file.read()
pubInputs = json.loads(public_data)

pubParameters  = {
    "pubInput": [],
}
for pubInput in pubInputs:
    pubParameters["pubInput"].append(int(pubInput))

print("pubParameters", pubParameters)

pubParameters {'pubInput': [396]}


In [30]:
fullCircomInput = {**inputParameters, **proofParameters, **pubParameters}

with open('fixtures/full-circom-input-0.json', 'w') as outfile:
    json.dump(fullCircomInput, outfile)


In [34]:
p = curve.field_modulus
r = curve.curve_order # baby jubjub prime
x = 4965661367192848881 # parameter x = 0x44e992b44a6909f1
assert p == 36*x**4 + 36*x**3 + 24*x**2 + 6*x + 1
assert r == 36*x**4 + 36*x**3 + 18*x**2 + 6*x + 1
assert p % 4 == 3
assert p % 6 == 1

In [16]:
k = 5
n = math.ceil(254 / k)
print(n)
math.log((18*k)**2 * (2*k-1), 2) + 4*n 

51


220.15363119410165

In [6]:
def print_fq12_frobenius_coeff(n, k):
    gamma = [[0]*6]*12
    for j in range(12):
        gamma[j] = [ FQ2([9,1]) ** ( (i*(p**j-1)//6) % (p**2-1) ) for i in range(6)]
    for j in range(12):
        for i in range(6):
            A, B = gamma[j][i].coeffs
            a = A.n
            b = B.n
            for r in range(k):
                print(f"coeff[{j}][{i}][0][{r}] = {a%(2**n)};")
                a //= 2**n
            assert a == 0
            print("")
            for r in range(k):
                print(f"coeff[{j}][{i}][1][{r}] = {b%(2**n)};")
                b //= 2**n
            assert b == 0
            print("")

def Fp12convert(X, n, k):
    basis1 = X.coeffs
    ret = []
    for i in range(6):
        fq2elt = FQ2([basis1[i].n, 0]) + FQ2([basis1[i+6].n, 0]) * FQ2([9,1])
        ret.append([ numberToArray(fq2elt.coeffs[0].n, n, k) , numberToArray(fq2elt.coeffs[1].n, n, k) ])
    return ret

def printFQ12(X, n, k):
    in62 = Fp12convert(X, n, k)
    print("[")
    for i in range(len(in62)):
        print("[", end="")
        C = in62[i]
        for j in range(len(C)):
            print("[", end="")
            A = C[j]
            for idx in range(len(A)):
                print(f'"{A[idx]}"', end="")
                if idx != len(A)-1:
                    print(",")
            print("]", end="")
            if j != len(C)-1:
                print(",")
        print("]", end="")
        if i != len(in62)-1:
            print(",")
    print("]")

In [5]:
# print_fq12_frobenius_coeff(51,5)

coeff[0][0][0][0] = 1;
coeff[0][0][0][1] = 0;
coeff[0][0][0][2] = 0;
coeff[0][0][0][3] = 0;
coeff[0][0][0][4] = 0;

coeff[0][0][1][0] = 0;
coeff[0][0][1][1] = 0;
coeff[0][0][1][2] = 0;
coeff[0][0][1][3] = 0;
coeff[0][0][1][4] = 0;

coeff[0][1][0][0] = 1;
coeff[0][1][0][1] = 0;
coeff[0][1][0][2] = 0;
coeff[0][1][0][3] = 0;
coeff[0][1][0][4] = 0;

coeff[0][1][1][0] = 0;
coeff[0][1][1][1] = 0;
coeff[0][1][1][2] = 0;
coeff[0][1][1][3] = 0;
coeff[0][1][1][4] = 0;

coeff[0][2][0][0] = 1;
coeff[0][2][0][1] = 0;
coeff[0][2][0][2] = 0;
coeff[0][2][0][3] = 0;
coeff[0][2][0][4] = 0;

coeff[0][2][1][0] = 0;
coeff[0][2][1][1] = 0;
coeff[0][2][1][2] = 0;
coeff[0][2][1][3] = 0;
coeff[0][2][1][4] = 0;

coeff[0][3][0][0] = 1;
coeff[0][3][0][1] = 0;
coeff[0][3][0][2] = 0;
coeff[0][3][0][3] = 0;
coeff[0][3][0][4] = 0;

coeff[0][3][1][0] = 0;
coeff[0][3][1][1] = 0;
coeff[0][3][1][2] = 0;
coeff[0][3][1][3] = 0;
coeff[0][3][1][4] = 0;

coeff[0][4][0][0] = 1;
coeff[0][4][0][1] = 0;
coeff[0][4][0][2] = 0;
coe

In [10]:
import random
rand_twelve = []
for i in range(12):
    rand_twelve.append(random.randrange(p))
X = FQ12(rand_twelve)

In [38]:
Phi12 = p**4 - p**2 + 1
lamb3 = 1
lamb2 = 6*x**2 + 1
lamb1 = -36*x**3 - 18*x**2 - 12*x + 1
lamb0 = -36*x**3 - 30*x**2 - 18*x - 2
Phi12 // r == lamb3 * p**3 + lamb2 * p**2 + lamb1 * p + lamb0 

True

In [43]:
Y = X ** ((p**6 - 1) * (p**2 + 1))

In [56]:
Z = Y ** (Phi12 // r) 

In [75]:
printFQ12(Z, 51, 5)

[
[["1230240385392124",
"357936908694134",
"319960135806541",
"67137439243720",
"281329545235137"],
["1677005579044279",
"105732828588556",
"2020928832000090",
"578203068157553",
"183901047932913"]],
[["230625234712896",
"786959675586198",
"1746080908042125",
"403584908312062",
"813495851716921"],
["1536845135038187",
"1354402396345912",
"376661622282677",
"2071001058307412",
"347839451041051"]],
[["389046412837792",
"467896831404857",
"36908224715553",
"1956642825555426",
"763035301011827"],
["2222725870352709",
"1430217087684263",
"1847503954532002",
"2147810405186655",
"842092461234304"]],
[["1387452211493618",
"928428321931967",
"2038052718346330",
"556014289076437",
"552700864892979"],
["1196299509241438",
"1838563232074719",
"1192057449525785",
"920676941249199",
"94373610719323"]],
[["1193072661480307",
"1397585762870644",
"2230742103912463",
"1318819829618615",
"773527463998786"],
["2066730672277490",
"1206950648341309",
"1132571467697697",
"1377609580293480",
"552765435294739"

In [72]:
x.bit_length()

63